In [57]:
import numpy
import scipy.special
import scipy.ndimage
import matplotlib.pyplot
import imageio
import glob
import csv
import cmath
import math
import time

In [58]:
class neuralNetwork:
    

    def __init__(self, inputnodes, hiddennodes1, hiddennodes2, outputnodes, learningrate):

        self.inodes = inputnodes
        self.hnodes1 = hiddennodes1
        self.hnodes2 = hiddennodes2
        self.onodes = outputnodes
        
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes1, self.inodes))
        self.wih = numpy.array(self.wih, ndmin=2, dtype='complex128')
        self.wih += 1j * numpy.random.normal(0.0,pow(self.inodes, -0.5), (self.hnodes1, self.inodes))

        self.whh = numpy.random.normal(0.0, pow(self.hnodes1, -0.5), (self.hnodes2, self.hnodes1))
        self.whh = numpy.array(self.whh, ndmin=2, dtype='complex128')
        self.whh += 1j * numpy.random.normal(0.0,pow(self.hnodes1, -0.5), (self.hnodes2, self.hnodes1))

        self.who = numpy.random.normal(0.0, pow(self.hnodes2, -0.5), (self.onodes, self.hnodes2))
        self.who = numpy.array(self.who, ndmin=2, dtype='complex128')
        self.who += 1j * numpy.random.normal(0.0, pow(self.hnodes2, -0.5), (self.onodes, self.hnodes2))

        self.lr = learningrate
        
        #self.activation_function = lambda x: numpy.minimum(numpy.maximum(0, x), 1)
        #self.activation_function = lambda x: numpy.maximum(0, x)
        #self.activation_function = lambda x: numpy.maximum(a*x,x)
        #a = 0.01
        self.activation_function = lambda x: numpy.exp(x) / numpy.sum(numpy.exp(x))
        
        
    def train(self, inputs_list, targets_list):
        
        inputs = numpy.array(inputs_list, dtype=complex, ndmin=2).T
        targets = numpy.array(targets_list, dtype=complex, ndmin=2).T

        hidden1_inputs = numpy.dot(self.wih, inputs)
        hidden1_act = self.activation_function(numpy.linalg.norm(hidden1_inputs))
        hidden1_outputs_real = hidden1_act * numpy.cos(numpy.angle(hidden1_inputs))
        hidden1_outputs_imag = hidden1_act * numpy.sin(numpy.angle(hidden1_inputs))
        hidden1_outputs = hidden1_outputs_real + 1j * hidden1_outputs_imag

        hidden2_inputs = numpy.dot(self.whh, hidden1_outputs)
        hidden2_act = self.activation_function(numpy.linalg.norm(hidden2_inputs))
        hidden2_outputs_real = hidden2_act * numpy.cos(numpy.angle(hidden2_inputs))
        hidden2_outputs_imag = hidden2_act * numpy.sin(numpy.angle(hidden2_inputs))
        hidden2_outputs = hidden2_outputs_real + 1j * hidden2_outputs_imag

        final_inputs = numpy.dot(self.who, hidden2_outputs)
        final_outputs_real = self.activation_function(final_inputs.real)
        final_outputs_imag = self.activation_function((-1j * final_inputs.imag).real)
        final_outputs = final_outputs_real #+ 1j * final_outputs_imag

        output_errors = targets - final_outputs
        hidden2_errors = numpy.dot(self.who.T, output_errors)
        hidden1_errors = numpy.dot(self.whh.T, hidden2_errors)

        self.who += self.lr * numpy.dot(output_errors * (1.0 - numpy.square(final_outputs)), numpy.conj(hidden2_outputs).T)
        self.whh += self.lr * numpy.dot(hidden2_errors * (1.0 - numpy.square(hidden2_outputs)), numpy.conj(hidden1_outputs).T)
        self.wih += self.lr * numpy.dot(hidden1_errors * (1.0 - numpy.square(hidden1_outputs)), numpy.conj(inputs).T)

    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, dtype=complex, ndmin=2).T

        hidden1_inputs = numpy.dot(self.wih, inputs)
        hidden1_act = self.activation_function(numpy.linalg.norm(hidden1_inputs))
        hidden1_outputs_real = hidden1_act * numpy.cos(numpy.angle(hidden1_inputs))
        hidden1_outputs_imag = hidden1_act * numpy.sin(numpy.angle(hidden1_inputs))
        hidden1_outputs = hidden1_outputs_real + 1j * hidden1_outputs_imag

        hidden2_inputs = numpy.dot(self.whh, hidden1_outputs)
        hidden2_act = self.activation_function(numpy.linalg.norm(hidden2_inputs))
        hidden2_outputs_real = hidden2_act * numpy.cos(numpy.angle(hidden2_inputs))
        hidden2_outputs_imag = hidden2_act * numpy.sin(numpy.angle(hidden2_inputs))
        hidden2_outputs = hidden2_outputs_real + 1j * hidden2_outputs_imag

        final_inputs = numpy.dot(self.who, hidden2_outputs)
        final_outputs_real = self.activation_function(final_inputs.real)
        final_outputs_imag = self.activation_function((-1j * final_inputs.imag).real)
        final_outputs = final_outputs_real #+ 1j * final_outputs_imag
        
        return final_outputs_real

In [65]:
input_nodes = 784
hidden_nodes1 = 200
hidden_nodes2 = 100
output_nodes = 10

learning_rate = 0.01

n = neuralNetwork(input_nodes, hidden_nodes1, hidden_nodes2, output_nodes, learning_rate)

training_data_file = open("mnist_train_100.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

print("Start Training..")

epochs = 20

t_s = time.time()
for e in range(epochs):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        
    print("epochs:", e+1, "/", epochs)
t_f = time.time()

print("timecost = {0:0.2f} sec".format(t_f-t_s))

test_data_file = open("mnist_train_100.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

scorecard = []

for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    outputs = n.query(inputs)
    label = numpy.argmax(outputs)
    if label == correct_label:
        scorecard.append(1)
    else:
        scorecard.append(0)
    
scorecard_array = numpy.asarray(scorecard)
print("Performance =", scorecard_array.sum() / scorecard_array.size)

Start Training..
epochs: 1 / 20
epochs: 2 / 20
epochs: 3 / 20
epochs: 4 / 20
epochs: 5 / 20
epochs: 6 / 20
epochs: 7 / 20
epochs: 8 / 20
epochs: 9 / 20
epochs: 10 / 20
epochs: 11 / 20
epochs: 12 / 20
epochs: 13 / 20
epochs: 14 / 20
epochs: 15 / 20
epochs: 16 / 20
epochs: 17 / 20
epochs: 18 / 20
epochs: 19 / 20
epochs: 20 / 20
timecost = 6.57 sec
Performance = 1.0


In [ ]:
Start Training..
epochs: 1 / 1
timecost = 0.31 sec
Performance = 0.49088333333333334

In [ ]:
Start Training..
epochs: 1 / 5
epochs: 2 / 5
epochs: 3 / 5
epochs: 4 / 5
epochs: 5 / 5
timecost = 1.66 sec
Performance = 0.6612833333333333

In [ ]:
Start Training..
epochs: 1 / 10
epochs: 2 / 10
epochs: 3 / 10
epochs: 4 / 10
epochs: 5 / 10
epochs: 6 / 10
epochs: 7 / 10
epochs: 8 / 10
epochs: 9 / 10
epochs: 10 / 10
timecost = 3.60 sec
Performance = 0.6586666666666666

In [ ]:
Start Training..
epochs: 1 / 20
epochs: 2 / 20
epochs: 3 / 20
epochs: 4 / 20
epochs: 5 / 20
epochs: 6 / 20
epochs: 7 / 20
epochs: 8 / 20
epochs: 9 / 20
epochs: 10 / 20
epochs: 11 / 20
epochs: 12 / 20
epochs: 13 / 20
epochs: 14 / 20
epochs: 15 / 20
epochs: 16 / 20
epochs: 17 / 20
epochs: 18 / 20
epochs: 19 / 20
epochs: 20 / 20
timecost = 7.56 sec
Performance = 0.66585

In [ ]:
Start Training..
epochs: 1 / 20
epochs: 2 / 20
epochs: 3 / 20
epochs: 4 / 20
epochs: 5 / 20
epochs: 6 / 20
epochs: 7 / 20
epochs: 8 / 20
epochs: 9 / 20
epochs: 10 / 20
epochs: 11 / 20
epochs: 12 / 20
epochs: 13 / 20
epochs: 14 / 20
epochs: 15 / 20
epochs: 16 / 20
epochs: 17 / 20
epochs: 18 / 20
epochs: 19 / 20
epochs: 20 / 20
timecost = 6.57 sec
Performance = 1.0

train = 100
test = 100

In [55]:
import numpy
import scipy.special
import scipy.ndimage
import matplotlib.pyplot
import imageio
import glob
import csv
import cmath
import math
import time


class neuralNetwork:
    
    def __init__(self, inputnodes, hiddennodes1,hiddennodes2 ,outputnodes,learningrate, r,theta1, theta2, theta3):

        self.inodes = inputnodes
        self.hnodes1 = hiddennodes1
        self.hnodes2 = hiddennodes2
        self.onodes = outputnodes
        
        self.theta1 = theta1
        self.theta2 = theta2
        self.theta3 = theta3
        self.r = r
        self.lr = learningrate
        
        self.theta1 = numpy.random.uniform(0, 2 * numpy.pi, (self.hnodes1, self.inodes))
        self.theta2 = numpy.random.uniform(0, 2 * numpy.pi, (self.hnodes2, self.hnodes1))
        self.theta3 = numpy.random.uniform(0, 2 * numpy.pi, (self.onodes, self.hnodes2))
        self.r = numpy.random.normal(0.0)
    
        self.wih = numpy.exp(1j*self.theta1)
        self.wih = numpy.array(self.wih, ndmin=2, dtype='complex128')
        self.wih *= self.r
        
        self.whh = numpy.exp(1j*self.theta2)
        self.whh = numpy.array(self.whh, ndmin=2, dtype='complex128')
        self.whh *= self.r
        
        self.who = numpy.exp(1j*self.theta3)
        self.who = numpy.array(self.who, ndmin=2, dtype='complex128')
        self.who *= self.r
        
        self.activation_function = lambda x: numpy.exp(x) / numpy.sum(numpy.exp(x))
        
        pass

    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, dtype=complex, ndmin=2).T
        targets = numpy.array(targets_list, dtype=complex, ndmin=2).T

        hidden1_inputs = numpy.dot(self.wih, inputs)
        hidden1_act = self.activation_function(numpy.linalg.norm(hidden1_inputs))
        hidden1_outputs_real = hidden1_act * numpy.cos(numpy.angle(hidden1_inputs))
        hidden1_outputs_imag = hidden1_act * numpy.sin(numpy.angle(hidden1_inputs))
        hidden1_outputs = hidden1_outputs_real + 1j * hidden1_outputs_imag

        hidden2_inputs = numpy.dot(self.whh, hidden1_outputs)
        hidden2_act = self.activation_function(numpy.linalg.norm(hidden2_inputs))
        hidden2_outputs_real = hidden2_act * numpy.cos(numpy.angle(hidden2_inputs))
        hidden2_outputs_imag = hidden2_act * numpy.sin(numpy.angle(hidden2_inputs))
        hidden2_outputs = hidden2_outputs_real + 1j * hidden2_outputs_imag

        final_inputs = numpy.dot(self.who, hidden2_outputs)
        final_outputs_real = self.activation_function(final_inputs.real)
        final_outputs_imag = self.activation_function((-1j * final_inputs.imag).real)
        final_outputs = final_outputs_real 

        output_errors = targets - final_outputs
        hidden2_errors = numpy.dot(self.who.T, output_errors)
        hidden1_errors = numpy.dot(self.whh.T, hidden2_errors)

        self.who += self.lr * numpy.dot(output_errors * (1.0 - numpy.square(final_outputs)), numpy.conj(hidden2_outputs).T)
        self.whh += self.lr * numpy.dot(hidden2_errors * (1.0 - numpy.square(hidden2_outputs)), numpy.conj(hidden1_outputs).T)
        self.wih += self.lr * numpy.dot(hidden1_errors * (1.0 - numpy.square(hidden1_outputs)), numpy.conj(inputs).T)
        
        pass

    def query(self, inputs_list):
        
        inputs = numpy.array(inputs_list, dtype=complex, ndmin=2).T

        hidden1_inputs = numpy.dot(self.wih, inputs)
        hidden1_act = self.activation_function(numpy.linalg.norm(hidden1_inputs))
        hidden1_outputs_real = hidden1_act * numpy.cos(numpy.angle(hidden1_inputs))
        hidden1_outputs_imag = hidden1_act * numpy.sin(numpy.angle(hidden1_inputs))
        hidden1_outputs = hidden1_outputs_real + 1j * hidden1_outputs_imag

        hidden2_inputs = numpy.dot(self.whh, hidden1_outputs)
        hidden2_act = self.activation_function(numpy.linalg.norm(hidden2_inputs))
        hidden2_outputs_real = hidden2_act * numpy.cos(numpy.angle(hidden2_inputs))
        hidden2_outputs_imag = hidden2_act * numpy.sin(numpy.angle(hidden2_inputs))
        hidden2_outputs = hidden2_outputs_real + 1j * hidden2_outputs_imag

        final_inputs = numpy.dot(self.who, hidden2_outputs)
        final_outputs_real = self.activation_function(final_inputs.real)
        final_outputs_imag = self.activation_function((-1j * final_inputs.imag).real)
        final_outputs = final_outputs_real 
        
        return final_outputs_real


input_nodes = 784
hidden_nodes1 = 200
hidden_nodes2 = 100
output_nodes = 10

learning_rate = 0.01


r = 1
theta_1 = numpy.pi/4
theta_2 = numpy.pi/3
theta_3 = numpy.pi/2


n = neuralNetwork(input_nodes,hidden_nodes1,hidden_nodes2, output_nodes, learning_rate,r, theta_1, theta_2, theta_3)

training_data_file = open("mnist_train_100.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

print("Start Training..")

epochs = 5

t_s = time.time()
for e in range(epochs):
    for record in training_data_list:
        all_values = record.split(',')
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        
    print("epochs:", e+1, "/", epochs)
t_f = time.time()

print("timecost = {0:0.2f} sec".format(t_f-t_s))

test_data_file = open("mnist_train_100.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

scorecard = []

for record in test_data_list:
    all_values = record.split(',')
    correct_label = int(all_values[0])
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    outputs = n.query(inputs)
    label = numpy.argmax(outputs)
    if label == correct_label:
        scorecard.append(1)
    else:
        scorecard.append(0)
    
scorecard_array = numpy.asarray(scorecard)
print("Performance =", scorecard_array.sum() / scorecard_array.size)

Start Training..
epochs: 1 / 5


<ipython-input-55-ec23590a6476>:45: RuntimeWarning: overflow encountered in exp
  self.activation_function = lambda x: numpy.exp(x) / numpy.sum(numpy.exp(x))
<ipython-input-55-ec23590a6476>:45: RuntimeWarning: invalid value encountered in double_scalars
  self.activation_function = lambda x: numpy.exp(x) / numpy.sum(numpy.exp(x))


epochs: 2 / 5
epochs: 3 / 5
epochs: 4 / 5
epochs: 5 / 5
timecost = 1.87 sec
Performance = 0.13


In [ ]:
Start Training..
epochs: 1 / 1
timecost = 0.33 sec
Performance = 0.62

In [ ]:
Start Training..
epochs: 1 / 2
epochs: 2 / 2
timecost = 0.64 sec
Performance = 0.98

In [ ]:
Start Training..
epochs: 1 / 3
epochs: 2 / 3
epochs: 3 / 3
timecost = 1.01 sec
Performance = 1.0

In [ ]:
Start Training..
"""""<ipython-input-54-59e867d95084>:45: RuntimeWarning: overflow encountered in exp
  self.activation_function = lambda x: numpy.exp(x) / numpy.sum(numpy.exp(x))
<ipython-input-54-59e867d95084>:45: RuntimeWarning: invalid value encountered in double_scalars
  self.activation_function = lambda x: numpy.exp(x) / numpy.sum(numpy.exp(x))"""
epochs: 1 / 4
epochs: 2 / 4
epochs: 3 / 4
epochs: 4 / 4
timecost = 1.29 sec
Performance = 0.13